In [2]:
import Pkg; Pkg.add("Cbc")
using Pkg, JuMP, Plots, Cbc, GraphRecipes, LightGraphs, SimpleWeightedGraphs, SparseArrays

Error: Canceled future for execute_request message before replies were done

### ECE/CS/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2022 ###


# Madison Bus Route Designing: Preliminary Report

Team Members: Rabeeya Hamid, Ai Guan, Naman Gupta

Data: 04/22/2022

*****

### Table of Contents

1. [Problem Description](#1.-Problem-Description)
2. [Data Collection](#2.-Data-Collection)
3. [Mathematical Model](#3.-Mathematical-Model)
4. [Preliminary Code Implementation](#4.-Code-Implementation)
5. [Preliminary Results and Analysis](#5.-Results-and-Analysis)


## 1. Problem Description


Based on the increasing need of public transportation, the City of Madison Metro Transit is proposing a major network redesign ([Transit Network Redesign](https://www.cityofmadison.com/metro/routes-schedules/transit-network-redesign)). Currently, they are surveying the students of UW-Madison and the residents of Madison for suggestions on their travel preferences, so that they can design the route system to better meet user demand and business. As University of Wisconsin-Madison students, our team thinks the redesign project would also have significant impact on our daily lives. 

The aim of the project is to model the routing design as a network optimization problem, so that we can propose the model as a part of their feedback survey. The family of network optimization problem includes assignment, critical path, max flow, shortest path, transportation, and min-cost flow problems . One of the primary results of the model is a network of arcs and nodes that illustrate the optimal routing among the points ([Network Optimization](http://home.ubalt.edu/ntsbarsh/opre640a/partIII.htm#:~:text=The%20family%20of%20network%20optimization,called%20a%20vertex%2C%20or%20point)).

The project is a transportation problem in the network optimization problem family, and the nodes represent the bus stations, and the arcs represent the transportation flow between two stations. The model consist of three parts: decision variables, constraints, and objective function. We would like to model the optimal routes to meet the demand of ridership for each bus stop in Madison. Therefore, the objective function of the problem to maximizing the number of passengers boarding for each bus with potential constraints including fuel limitation, bus capacity, routing regulations and bus station demand. 

## 2. Data Collection

The data utilized in the project is the publicly available data from the City of Madison website. Two datasets are selected corresponding to the 
   1.	Metro Transit Ridership by Stop | City of Madison Open Data.  [Data Source](https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::metro-transit-ridership-by-stop)
   2.	Metro Transit Bus Route Patterns | City of Madison Open Data. [Data Source](https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::metro-transit-bus-route-patterns/about)
    
The first dataset contains a list of all bus stops, their position geographically (longitude and latitude) and the ridership at that bus stop averaged over 12 days. We do not have distances between these bus stops, nor do we have connections between them, for which we propose the following approach. 
   *   We calculate the Manhattan distance between the geographic position of the bus stops to define the distances between them. 
   *   These distances will be multiplied by a constant of proportionality to determine the fuel cost associated with each 
        path. 
The second dataset contains a list of present bus routes, the bus stops along the routes and the total ridership for the bus routes averaged over 12 days. 
	
For the preliminary report, the focus is to generate the illustrative demo of the model. It is a smaller bus network which consists of 8 bus stations.

In [ ]:
# Add packages
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")

using CSV
using DataFrames

# import data set as a dataframe
ridership_data = CSV.read("data/Metro_Transit_Ridership_by_Stop.csv", DataFrame)
route_patterns_data = CSV.read("data/Metro_Transit_Bus_Route_Patterns.csv", DataFrame)
stops_data = CSV.read("data/Metro_Transit_Bus_Stops.csv", DataFrame);

In [ ]:
first(ridership_data, 5)

In [ ]:
first(route_patterns_data, 5)

In [ ]:
first(stops_data, 5)

In [ ]:
N = size(ridership_data,1)

In [ ]:
import Pkg; Pkg.add("Geodesy")
using Geodesy

stops = Vector{LatLon{Float64}}()

for i in eachrow(ridership_data)
    x = LatLon(i.Lat, i.Lon)
    push!(stops, x)
end

distances = zeros(N,N)

for i in 1:N
    for j in 1:N
        distances[i,j] = euclidean_distance(stops[i], stops[j])
    end
end


In [ ]:
distances

In [ ]:
stop_route = Dict()

for i in eachrow(stops_data)
    if i.Route != "None"
        stop_route[i.stop_id] = i.Route
    end
end

stop_route

## 3. Mathematical Model

### Parameters Given:
- Number of bus stops ```n``` : number of nodes in our network
- Adjacency Matrix ```Adj``` ($n \times n$) : describes connections between bus stops
- Distance Matrix ```D``` ($n \times n$): Euclidean distances between all bus stops
- Ridership vector ```r```($n \times 1$): the ridership demand at every bus stop
- Start node ```n_i```: the bus stop from where our bus starts
- End node ```n_e```: the bus stop where our bus route ends
- Distance (Fuel) limit ```f```: the distance our bus can travel in total

In [15]:
n = 8

Adj = [0 1 0 0 1 0 1 0;
       0 0 1 0 1 1 0 0;
       1 1 0 1 0 1 1 0;
       0 0 0 0 1 0 0 1;
       1 1 1 0 0 1 1 0;
       0 0 0 1 0 0 0 1;
       1 1 1 0 1 0 0 1;
       1 0 1 1 0 0 1 0;]

D = [0 4 6 3 10 1 8 10;
     5 0 2 6 5 5 4 6;
     1 8 0 4 2 7 2 3;
     7 2 3 0 4 2 8 9;
     2 3 9 4 0 5 1 2;
     1 4 1 3 10 0 8 8;
     5 8 2 6 5 2 0 6;
     1 8 7 4 2 1 2 0]

r = [4, 1, 8, 2, 9, 10, 5, 8]

n_i = 2
n_e = 7

f = 20
;

### Plotting network:

In [16]:
edgelabel_dict = Dict()
for i in 1:n
    for j in 1:n
        edgelabel_dict[(i, j)] = D[i, j]
    end
end

membership = [1,2,1,1,1,1,3,1]
nodecolor = [colorant"lightseagreen", colorant"orange", colorant"yellow"]
# membership color
nodefillc = nodecolor[membership]

graph = graphplot(Adj, names="BS ".*string.(1:n), curvature_scalar=0.01, edgelabel=edgelabel_dict, markercolor=nodefillc)

LoadError: LoadError: UndefVarError: @colorant_str not defined
in expression starting at In[16]:9

_Orange: Start node <br>
Yellow: End node_

### Parameters Calculated:
- Number of paths ```e```: number of edges in our network 
- Incidence matrix ```Inc```($n \times e$): describes the edges in our network.
- Edge costs ```d_e```($e \times 1$): the distance corresponding to edges in the network
- Rider intake ```r_e```($e \times 1$): the riders taken up after traveling to each edge
- Flow vector ```b``` ($n \times 1$): includes the start and end node

### Function for incidence matrix:
Defining function to calculate incidence matrix from adjacency matrix, taken from: https://github.com/sbromberger/LightGraphs.jl/issues/1489

In [4]:
function my_incidence_matrix(G)
    I = vcat([src(e) for e in edges(G)], [dst(e) for e in edges(G)])
    J = vcat(collect(1:ne(G)), collect(1:ne(G)))
    V = vcat(fill(-1, ne(G)), fill(1, ne(G)))
    return sparse(I, J, V)
end
;

### Calculating incidence matrix and number of edges:

In [5]:
dirGraph = LightGraphs.DiGraph(Adj) # converting adjacency matrix into a directed graph

Inc = Array(my_incidence_matrix(dirGraph)) # converting directed graph to an incidence matrix

e = size(Inc)[2] # number of edges
;

LoadError: UndefVarError: LightGraphs not defined

### Defining edge costs, rider intake, and flow vector:

In [6]:
temp_D = D.*Adj
temp_r = zeros((n,n))

for i = 1:n
        temp_r[i, :] = r.*Adj[i, :]
end

d_e = zeros(0)
r_e = zeros(0)

for i = 1:n
    for j = 1:n
        if temp_D[i, j] != 0
            append!(d_e, temp_D[i, j])
        end
        if temp_r[i, j] != 0
            append!(r_e, temp_r[i, j])
        end
    end    
end

b = zeros(n)
b[n_i] = -1
b[n_e] = 1
;


**Decision variable:** <br>

$x$: all possible edges in the network, length = $e$ <br>
    $$ x =   \left\{
\begin{array}{ll}
      1 & \text{If the bus travels on that edge}  \\
      0 & \text{If the bus doesn NOT travel on that edge} \\
\end{array} 
\right.  $$

**Objective function:** <br>
$$ \max_{x} \text{    }r_e \cdot x$$

**Constraints:** <br>
  s.t. 
$$ \text{Inc}*x = b, $$
$$ d_e \cdot x <= f  $$


## 4. Code Implementation

In [7]:
bus_route = Model(with_optimizer(Cbc.Optimizer))

@variable(bus_route, x[1:e], Bin) # binary decision variable x that maps out the [ath]
@constraint(bus_route, Inc*x .== b) # nodal conservation
@constraint(bus_route, dot(d_e, x) <= f) # fuel constraint

@objective(bus_route, Max, dot(r_e, x))  # maximize people getting on the bus
    
optimize!(bus_route) 

LoadError: UndefVarError: Cbc not defined

## 5. Results and Analysis

### Defining dictionary for converting edge index to path:

In [8]:
edgeIndex2path = Dict()
count = 1
for i = 1:n
    for j = 1:n
        if Adj[i, j] != 0
            edgeIndex2path[count] = (i, j)
            count = count + 1
        end
    end
end

### Finding the optimal path taken:

In [9]:
opt_path = value.(x)

# finding edges travelled using edgeIndex2path dictionary
path_taken_x = zeros(0)
path_taken_y = zeros(0)
for i = 1:e
    if opt_path[i] != 0
        temp = edgeIndex2path[i]
        append!(path_taken_x, temp[1] )
        append!(path_taken_y, temp[2] )
    end
end

# defining dictionary with 1 for travelled edges, 0 for untravelled edges
travelled_edges_dict = Dict()
for k = 1:length(path_taken_x)
    for i in 1:n
        for j in 1:n
            if i == path_taken_x[k] && j == path_taken_y[k] 
                travelled_edges_dict[(i, j)] = 1
            elseif haskey(travelled_edges_dict, (i, j))
                if travelled_edges_dict[(i, j)] == 1
                    travelled_edges_dict[(i, j)] = 1
                end
            else 
                travelled_edges_dict[(i, j)] = 0
            end
        end
    end
end

# printing the edges travelled on

print("The bus travelled on these edges: \n")
for i = 1:n
    for j = 1:n
        if travelled_edges_dict[(i, j)] != 0
            println((i, j))
        end
    end
end

LoadError: UndefVarError: x not defined

### Finding nodes travelled:

In [10]:
nodes = 1:n

temp_path = zeros((n,n))

# defining temporary variable temp_path to store nodes travelled to for on each edge index
# (zeros need to be filtered out) 
for i = 1:n
        temp_path[i, :] = nodes.*Adj[i, :]
end

path = zeros(0)

# filtering out zeros
for i = 1:n
    for j = 1:n
        if temp_path[i, j] != 0
            append!(path, temp_path[i, j])
        end
    end    
end


# finding actual nodes travelled to using optimal edge vector x
temp_path = path.*opt_path

path = zeros(0)

# filtering out zeros again
for p in temp_path
    if p != 0
        append!(path, p)
    end
end

# defining membership based on nodes travelled to, 1 for not travelled to, 2 for travelled to
membership = ones(n)
for i = 1:n
    for p in path 
        if p == i
            membership[i] = 2
        end
    end
end

# start node is also travelled to 
membership[n_i] = 2

# convert to int for indexing
for i = 1:n
    membership[i] = convert(Int, membership[i])
end

# assigning colours, yellow for travelled to, seagreen for not
int_membership = Int.(membership)
nodecolor = [colorant"lightseagreen", colorant"orange", colorant"yellow"]
nodefillc = nodecolor[int_membership]
;

LoadError: UndefVarError: opt_path not defined

### Plotting final network:
Yellow bus stops: are included in route
Seagreen bus stops: are NOT included in route
Paths with 1: are included in route
Paths with 0: are NOT included in route

In [11]:
# edges are 1 for travelled on, 0 for not travelled on
graph = graphplot(Adj, names="BS ".*string.(1:n), curvature_scalar=0.11, edgelabel=travelled_edges_dict, markercolor=nodefillc)


LoadError: UndefVarError: travelled_edges_dict not defined

### Fuel used, and people picked up:

In [12]:
println("Fuel used: ", dot(d_e, opt_path))
println("People picked up: ", dot(r_e, opt_path)) 

LoadError: UndefVarError: dot not defined

# Next Steps